In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import mph
import pprint
from jpype.types import JInt, JDouble, JArray, JString

from pathlib import Path
import sys
sys.path.append(str(Path.cwd() / 'util/'))

from functions import *

config = load_config( "config/CoaxCavity.yaml" )

In [3]:
client = mph.start()
pymodel = client.create('CoaxCavity')

In [4]:
parameters = pymodel/'parameters'
(parameters/'Parameters 1').rename('Parameters')

for parameter in config['parameter']:
    value = config['parameter'][parameter]
    pymodel.parameter( parameter, value)

In [5]:
model = pymodel.java
comp1 = model.component().create("comp1", True)

geom1 = comp1.geom().create("geom1", 3) # 3 means 3D
geom1.lengthUnit("mm")
geom1.run()


boundary = geom1.create("boundary", "Block")
boundary.set("pos", ["-box_length/2", "-box_length/2", "outer_height - box_height"])
boundary.set("size", ["box_length", "box_length", "box_height"])
geom1.run("boundary")

# Make cavity box 
block1 = geom1.create("block1", "Block")
block1.set("pos", ["-box_length/2", "-box_length/2", "outer_height - box_height"])
block1.set("size", ["box_length", "box_length", "box_height"])
geom1.run("block1")

# Work plane 1
wp1 = geom1.create("wp1", "WorkPlane")
wp1.set("unite", True)
c1 = wp1.geom().create("c1", "Circle")
c1.set("pos",  [0., 0.] )
c1.set("r", 'outer_radius')
wp1.geom().run("c1")

# Extrude 1
ext1 = geom1.create("ext1", "Extrude")
ext1.set("workplane", "wp1")
ext1.selection("input").set("wp1")
ext1.setIndex("distance", 'outer_height', 0) # 0 means first row ?
geom1.run("ext1")

# Work plane 2
wp2 = geom1.create("wp2", "WorkPlane")
wp2.set("unite", True)
c2 = wp2.geom().create("c2", "Circle")
c2.set("pos",  [0., 0.] )
c2.set("r", 'coax_radius')
wp2.geom().run("c2")

# Extrude 2
ext2 = geom1.create("ext2", "Extrude")
ext2.set("workplane", "wp2")
ext2.selection("input").set("wp2")
ext2.setIndex("distance", 'coax_height', 0)
geom1.run("ext2")

# Subtract
diff1 = geom1.create("diff1", "Difference")
diff1.selection("input").set("block1")
diff1.selection("input2").set("ext1")
geom1.run("diff1")

# Union 1
uni1 = geom1.create("uni1", "Union")
uni1.selection("input").set("diff1", "ext2")
uni1.set("intbnd", False)
geom1.run("uni1")

# Work plane 6
wp6 = geom1.create("wp6", "WorkPlane")
wp6.set("unite", True)
wp6.set("quickz", "chip_pos_z - wafer_height")
r3 = wp6.geom().create("r3", "Rectangle")
r3.set("base", "center")
r3.set("pos", ['0', 'chip_pos_y'])
r3.set("size", ['wafer_width', 'wafer_length'])
wp6.geom().run("r3")
wp6.label("Chip")
geom1.run("wp6")

# Extrude 6
ext6 = geom1.create("ext6", "Extrude")
ext6.set("workplane", "wp6")
ext6.selection("input").set("wp6")
ext6.setIndex("distance", 'wafer_height', 0)
geom1.run("ext6")

# Work plane 7
wp7 = geom1.create("wp7", "WorkPlane")
wp7.set("unite", True)
wp7.set("quickz", "chip_pos_z")
r4 = wp7.geom().create("r4", "Rectangle")
r4.set("size", ['chip_width', 'chip_height'])
r4.set("base", "center")
r4.set("pos", ['0', 'chip_pos_y + chip_height/2 + chip_gap/2'])
wp7.geom().run("r4")

fil3 = wp7.geom().create("fil3", "Fillet")
fil3.selection("pointinsketch").set("r4", 1, 2, 3, 4)
fil3.set("radius", 'chip_fillet')
wp7.geom().run("fil3")

r5 = wp7.geom().create("r5", "Rectangle")
r5.set("base", "center")
r5.set("pos", ['0', 'chip_pos_y + jj_height/2 + finger_height/2'])
r5.set("size", ["jj_width", "finger_height"])
wp7.geom().run("r5")

# Union 2
uni2 = wp7.geom().create("uni2", "Union")
uni2.selection("input").set("fil3", "r5")
uni2.set("intbnd", False)
wp7.geom().run("uni2")

mir2 = wp7.geom().create("mir2", "Mirror")
mir2.selection("input").set("uni2")
mir2.set("pos", ['0', 'chip_pos_y'])
mir2.set("keep", True)
mir2.set("axis", [0., 1.])
wp7.geom().run("mir2")
wp7.label("Transmon pad")

r6 = wp7.geom().create("r6", "Rectangle")
r6.set("base", "center")
r6.set("size", ['jj_width','jj_height'])
r6.set("pos", ['0', 'chip_pos_y'])
wp7.geom().run("r6")

geom1.run("wp7")

<java object 'com.comsol.clientapi.impl.GeomSequenceClient'>

In [6]:
# run geometry and set view after making everything
geom1.run()
comp1.view("view1").set("transparency", True)

<java object 'com.comsol.clientapi.impl.ViewClient'>

In [7]:
mesh1 = comp1.mesh().create("mesh1")
emw  = comp1.physics().create("emw", "ElectromagneticWaves", "geom1")

In [8]:
std1 = model.study().create("std1")
sol1 = model.sol().create("sol1")
sol1.study("std1")

In [9]:
pymodel.save('mph/CoaxCavity.mph')
client.remove(pymodel)